# Variants in DNAJC13 and their Association with Parkinson's Disease Across Different Ancestral Backgrounds

* **Project**: gp2-hackathon-2023/Team 5 DNAJC13 Gene Assesment
* **Last updated**: September 2024
* **Version**: Python3.9
* **Data**: GP2 Release 7

# Summary
This notebook is focused on analysing association between DNAJC13 variants in East European (EAS) ancestry and Parkinson's disease.

# Imports

In [1]:
# Use the os package to interact with the environment
import os
import sys

# Bring in Pandas for Dataframe functionality
import pandas as pd
from functools import reduce

# Bring some visualization functionality 
import seaborn as sns

# numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

/tmp/ipykernel_30958/3943855956.py:26: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
    
def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for printing a query before executing it
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

In [ ]:
# Set up billing project and data path variables
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

## AMP-PD v2.5
## Explicitly define release v2.5 path 
AMP_CLINICAL_RELEASE_PATH = f'{AMP_RELEASE_PATH}/clinical'

AMP_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2021_v2-5release_0510/wgs'
AMP_WGS_RELEASE_PLINK_PATH = os.path.join(AMP_WGS_RELEASE_PATH, 'plink')
AMP_WGS_RELEASE_GATK_PATH = os.path.join(AMP_WGS_RELEASE_PATH, 'gatk')

## Print the information to check we are in the proper release and billing 
## This will be different for you, the user, depending on the billing project your workspace is on
print('Billing and Workspace')
print(f'Workspace Name @ `WORKSPACE_NAME`: {WORKSPACE_NAME}')
print(f'Billing Project @ `BILLING_PROJECT_ID`: {BILLING_PROJECT_ID}')
print(f'Workspace Bucket, where you can upload and download data @ `WORKSPACE_BUCKET`: {WORKSPACE_BUCKET}')
print('')

print('AMP-PD v2.5')
print(f'Path to AMP-PD v2.5 Clinical Data @ `AMP_CLINICAL_RELEASE_PATH`: {AMP_CLINICAL_RELEASE_PATH}')
print(f'Path to AMP-PD v2.5 WGS Data @ `AMP_WGS_RELEASE_PLINK_PATH`: {AMP_WGS_RELEASE_PLINK_PATH}')
print('')

## GP2 v7.0
## Explicitly define release v7.0 path 
GP2_CLINICAL_RELEASE_PATH = f'{GP2_RELEASE_PATH}/clinical_data'
GP2_META_RELEASE_PATH = f'{GP2_RELEASE_PATH}/meta_data'
GP2_SUMSTAT_RELEASE_PATH = f'{GP2_RELEASE_PATH}/summary_statistics'

GP2_RAW_GENO_PATH = f'{GP2_RELEASE_PATH}/raw_genotypes'
GP2_IMPUTED_GENO_PATH = f'{GP2_RELEASE_PATH}/imputed_genotypes'
print('GP2 v7.0')
print(f'Path to GP2 v7.0 Clinical Data @ `GP2_CLINICAL_RELEASE_PATH`: {GP2_CLINICAL_RELEASE_PATH}')
print(f'Path to GP2 v7.0 Metadata @ `GP2_META_RELEASE_PATH`: {GP2_META_RELEASE_PATH}')
print(f'Path to GP2 v7.0 Raw Genotype Data @ `GP2_RAW_GENO_PATH`: {GP2_RAW_GENO_PATH}')
print(f'Path to GP2 v7.0 Imputed Genotype Data @ `GP2_IMPUTED_GENO_PATH`: {GP2_IMPUTED_GENO_PATH}')

# Make working directory

In [4]:
ancestry = 'EAS'
WORK_DIR = f'202407_BURDEN_{ancestry}_R7'
WORKSPACE_DIR = f'202407_BURDEN_{ancestry}_R7'

! mkdir {WORK_DIR}

# Install Packages

## PLINK

In [5]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink; then
echo "Plink1.9 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink \n    -------"
wget -N http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 
unzip -o plink_linux_x86_64_20190304.zip
echo -e "\n plink downloaded and unzipped in /home/jupyter/tools \n "

fi

Plink1.9 is already installed in /home/jupyter/tools/


In [6]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink2; then
echo "Plink2 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink2 \n    -------"
wget -N https://s3.amazonaws.com/plink2-assets/plink2_linux_amd_avx2_20240704.zip
unzip -o plink2_linux_amd_avx2_20240704.zip
echo -e "\n plink2 downloaded and unzipped in /home/jupyter/tools \n "

fi

Plink2 is already installed in /home/jupyter/tools/


## ANNOVAR

In [7]:
%%bash

# Install ANNOVAR:
# https://www.openbioinformatics.org/annovar/annovar_download_form.php

if test -e /home/jupyter/tools/annovar; then

echo "annovar is already installed in /home/jupyter/tools/"
else
echo "annovar is not installed"
cd /home/jupyter/tools/

wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

tar xvfz annovar.latest.tar.gz

fi

annovar is already installed in /home/jupyter/tools/


In [8]:
%%bash

# Install ANNOVAR: Download resources for annotation

cd /home/jupyter/tools/annovar/

perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20140902 humandb/
#perl annotate_variation.pl -buildver hg38 -downdb cytoBand humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ensGene humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar exac03 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar avsnp147 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp30a humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar gnomad211_genome humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ljb26_all humandb/

NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGene.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneMrna.fa.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneVersion.txt.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished downloading annotation files for hg38 build version, with files saved at the 'humandb' directory
NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.idx.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished d

## RVTests

In [9]:
%%bash

#Install RVTESTS: Option 1 (~15min)
if test -e /home/jupyter/tools/rvtests; then

echo "rvtests is already installed"
else
echo "rvtests is not installed"

mkdir /home/jupyter/tools/rvtests
cd /home/jupyter/tools/rvtests

wget https://github.com/zhanxw/rvtests/releases/download/v2.1.0/rvtests_linux64.tar.gz 

tar -zxvf rvtests_linux64.tar.gz
fi

rvtests is already installed


In [10]:
# chmod to make sure you have permission to run the program
! chmod u+x /home/jupyter/tools/plink
! chmod u+x /home/jupyter/tools/plink2
! chmod 777 /home/jupyter/tools/rvtests/executable/rvtest

# Copy Over Files 

In [ ]:
## chr 3
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {GP2_IMPUTED_GENO_PATH}/{ancestry}/chr3_{ancestry}_release7.* {WORK_DIR}')

In [ ]:
## clinical data 

## master key
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {GP2_CLINICAL_RELEASE_PATH}/master_key_release7_final.csv {WORK_DIR}')

## related file 
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {GP2_META_RELEASE_PATH}/related_samples/{ancestry}_release7.related {WORK_DIR}')

In [ ]:
## PCs
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {GP2_RAW_GENO_PATH}/{ancestry}/{ancestry}_release7.eigenvec {WORK_DIR}')

In [ ]:
# refFlat
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {WORKSPACE_BUCKET}/notebooks/misc/refFlat_HG38_all_chr.txt {WORK_DIR}')

In [15]:
## Check files
! ls {WORK_DIR}

chr3_EAS_release7.log	chr3_EAS_release7.pvar	master_key_release7_final.csv
chr3_EAS_release7.pgen	EAS_release7.eigenvec	refFlat_HG38_all_chr.txt
chr3_EAS_release7.psam	EAS_release7.related


# Covariate File

In [ ]:
# Let's load the master key
key = pd.read_csv(f'{WORK_DIR}/master_key_release7_final.csv')
print(key.shape)
key.head()

In [ ]:
key = key[['GP2sampleID', 'baseline_GP2_phenotype', 'biological_sex_for_qc', 'age_at_sample_collection', 'age_of_onset', 'race_for_qc', 'label']]
key.rename(columns = {'GP2sampleID': 'IID',
                      'baseline_GP2_phenotype':'phenotype',
                                     'biological_sex_for_qc':'SEX', 
                                     'age_at_sample_collection':'AGE', 
                                     'race_for_qc':'RACE',
                                     'age_of_onset':'AAO'}, inplace = True)
key

In [ ]:
## Subset to keep ancestry of interest 
ancestry_key = key[key['label']==ancestry].copy()
ancestry_key.reset_index(drop=True)

In [ ]:
# Load information about related individuals 
related_df = pd.read_csv(f'{WORK_DIR}/{ancestry}_release7.related')
print(related_df.shape)
related_df

In [ ]:
# Make a list of just one set of related people
related_list = list(related_df['IID1'])

# Check value counts of related and remove only one related individual
ancestry_key = ancestry_key[~ancestry_key["IID"].isin(related_list)]

# Check size
print(ancestry_key.shape)
ancestry_key

In [21]:
# Convert phenotype to binary (0/1)
    # PD = 1; control = 0
pheno_mapping = {"PD": 1, "Control": 0}
ancestry_key['PHENO'] = ancestry_key['phenotype'].map(pheno_mapping).astype('Int64')

In [22]:
# Check value counts of pheno
ancestry_key['PHENO'].value_counts(dropna=False)

PHENO
1       2657
0       2572
<NA>      44
Name: count, dtype: Int64

In [ ]:
## Get the PCs
pcs = pd.read_csv(f'{WORK_DIR}/{ancestry}_release7.eigenvec', sep='\t')

# Split the single column into multiple columns (No longer needed with release 7)
#pcs_split = pcs['#FID,IID,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10'].str.split(',', expand=True)

selected_columns = ['IID', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5']


#pcs = pd.DataFrame(data=pcs_split.iloc[:, 1:7].values, columns=selected_columns)
pcs = pd.DataFrame(data=pcs.iloc[:, 1:7].values, columns=selected_columns)

# Drop the first row (since it's now the column names)
pcs = pcs.drop(0)

# Reset the index to remove any potential issues
pcs = pcs.reset_index(drop=True)


# Display the resulting DataFrame
print(pcs)

In [24]:
# Convert sex to binary (0/1)
    # Female = 1; male = 0
sex_mapping = {"Female": 1, "Male": 0}
ancestry_key['SEX'] = ancestry_key['SEX'].map(sex_mapping).astype('Int64')

In [25]:
# Check value counts of SEX
ancestry_key['SEX'].value_counts(dropna=False)

SEX
0       3388
1       1879
<NA>       6
Name: count, dtype: Int64

In [26]:
## Make covariate file
df = pd.merge(pcs, ancestry_key, on='IID', how='left')
df.columns

Index(['IID', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'phenotype', 'SEX', 'AGE',
       'AAO', 'RACE', 'label', 'PHENO'],
      dtype='object')

In [27]:
df['FID'] = 0

In [ ]:
## Clean up and keep columns we need 
final_df = df[['FID','IID', 'SEX', 'PHENO', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5']].copy()
final_df

In [29]:
final_df.groupby(['PHENO'])['SEX'].value_counts()

PHENO  SEX
0      0      1670
       1       675
1      0      1434
       1      1112
Name: count, dtype: int64

In [30]:
## Make file of sample IDs to keep 
samples_toKeep = final_df[['FID', 'IID']].copy()
samples_toKeep.to_csv(f'{WORK_DIR}/{ancestry}.samplestoKeep', sep = '\t', index=False, header=None)

In [31]:
## Save your covariate file
final_df.to_csv(f'{WORK_DIR}/{ancestry}_covariate_file.txt', sep = '\t', index=False)

In [32]:
## check to make sure file was created and saved
! ls {WORK_DIR}

chr3_EAS_release7.log	EAS_covariate_file.txt	master_key_release7_final.csv
chr3_EAS_release7.pgen	EAS_release7.eigenvec	refFlat_HG38_all_chr.txt
chr3_EAS_release7.psam	EAS_release7.related
chr3_EAS_release7.pvar	EAS.samplestoKeep


In [ ]:
# Save to workspace bucket (move from VM to workspace bucket)
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_covariate_file.txt {WORKSPACE_BUCKET}/{WORKSPACE_DIR}/')
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}.samplestoKeep {WORKSPACE_BUCKET}/{WORKSPACE_DIR}/')

In [ ]:
## Check workspace bucket
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} ls {WORKSPACE_BUCKET}/{WORKSPACE_DIR}//')

# Annotation using ANNOVAR

* *DNAJC13* from NCBI gene
* hg38 (chr3:132417502-132539032)

In [35]:
## extract region using plink

! /home/jupyter/tools/plink2 --pfile {WORK_DIR}/chr3_{ancestry}_release7 \
--chr 3 \
--from-bp 132417502  \
--to-bp 132539032 \
--recode vcf id-paste=iid \
--mac 2 \
--out {WORK_DIR}/{ancestry}_DNAJC13

PLINK v2.00a6LM AVX2 AMD (4 Jul 2024)          www.cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to 202407_BURDEN_EAS_R7/EAS_DNAJC13.log.
Options in effect:
  --chr 3
  --export vcf id-paste=iid
  --from-bp 132417502
  --mac 2
  --out 202407_BURDEN_EAS_R7/EAS_DNAJC13
  --pfile 202407_BURDEN_EAS_R7/chr3_EAS_release7
  --to-bp 132539032

Start time: Wed Jul 10 00:54:40 2024
7450 MiB RAM detected, ~4653 available; reserving 3725 MiB for main workspace.
Using up to 2 compute threads.
5136 samples (1868 females, 3268 males; 5136 founders) loaded from
202407_BURDEN_EAS_R7/chr3_EAS_release7.psam.
3353374 variants loaded from 202407_BURDEN_EAS_R7/chr3_EAS_release7.pvar.
1 binary phenotype loaded (2644 cases, 2452 controls).
Calculating allele frequencies... done.
562 variants removed due to allele frequency threshold(s)
(--maf/--max-maf/--mac/--max-mac).
1179 variants remaining after main filters.
--export vcf to 202407_BURDE

In [36]:
### Bgzip and Tabix (zip and index the file)
! bgzip -f {WORK_DIR}/{ancestry}_DNAJC13.vcf
! tabix -f -p vcf {WORK_DIR}/{ancestry}_DNAJC13.vcf.gz

In [37]:
## annotate using ANNOVAR
! perl /home/jupyter/tools/annovar/table_annovar.pl {WORK_DIR}/{ancestry}_DNAJC13.vcf.gz /home/jupyter/tools/annovar/humandb/ -buildver hg38 \
-out {WORK_DIR}/{ancestry}_DNAJC13.annovar \
-remove -protocol refGene,clinvar_20140902 \
-operation g,f \
--nopolish \
-nastring . \
-vcfinput


NOTICE: Running with system command <convert2annovar.pl  -includeinfo -allsample -withfreq -format vcf4 202407_BURDEN_EAS_R7/EAS_DNAJC13.vcf.gz > 202407_BURDEN_EAS_R7/EAS_DNAJC13.annovar.avinput>
NOTICE: Finished reading 1198 lines from VCF file
NOTICE: A total of 1179 locus in VCF file passed QC threshold, representing 1067 SNPs (740 transitions and 327 transversions) and 112 indels/substitutions
NOTICE: Finished writing allele frequencies based on 5480112 SNP genotypes (3800640 transitions and 1679472 transversions) and 575232 indels/substitutions for 5136 samples

NOTICE: Running with system command </home/jupyter/tools/annovar/table_annovar.pl 202407_BURDEN_EAS_R7/EAS_DNAJC13.annovar.avinput /home/jupyter/tools/annovar/humandb/ -buildver hg38 -outfile 202407_BURDEN_EAS_R7/EAS_DNAJC13.annovar -remove -protocol refGene,clinvar_20140902 -operation g,f --nopolish -nastring . -otherinfo>
-----------------------------------------------------------------
NOTICE: Processing operation=g pr

In [38]:
# Read in ANNOVAR multianno file
gene = pd.read_csv(f'{WORK_DIR}/{ancestry}_DNAJC13.annovar.hg38_multianno.txt', sep = '\t')
display(gene)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo5139,Otherinfo5140,Otherinfo5141,Otherinfo5142,Otherinfo5143,Otherinfo5144,Otherinfo5145,Otherinfo5146,Otherinfo5147,Otherinfo5148
0,3,132417545,132417545,G,A,UTR5,DNAJC13,NM_001329126:c.-17006G>A;NM_015268:c.-17006G>A,.,.,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
1,3,132417611,132417611,A,T,UTR5,DNAJC13,NM_001329126:c.-16940A>T;NM_015268:c.-16940A>T,.,.,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
2,3,132417746,132417746,C,T,UTR5,DNAJC13,NM_001329126:c.-16805C>T;NM_015268:c.-16805C>T,.,.,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
3,3,132417766,132417766,G,A,intronic,DNAJC13,.,.,.,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
4,3,132417820,132417820,G,-,intronic,DNAJC13,.,.,.,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1174,3,132538575,132538575,A,G,UTR3,DNAJC13,NM_001329126:c.*293A>G;NM_015268:c.*293A>G,.,.,...,0|0,0|1,1|1,0|0,0|0,0|0,0|0,0|0,0|1,0|0
1175,3,132538598,132538598,-,T,UTR3,DNAJC13,NM_001329126:c.*316_*317insT;NM_015268:c.*316_...,.,.,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
1176,3,132538679,132538679,A,T,UTR3,DNAJC13,NM_001329126:c.*397A>T;NM_015268:c.*397A>T,.,.,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
1177,3,132538755,132538755,C,A,UTR3,DNAJC13,NM_001329126:c.*473C>A;NM_015268:c.*473C>A,.,.,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0


In [39]:
## Filter intronic
intronic = gene[(gene['Func.refGene'] == 'intronic')]

In [40]:
## Filter UTR3
utr3 = gene[(gene['Func.refGene'] == 'UTR3')]

In [41]:
## Filter UTR5
utr5 = gene[(gene['Func.refGene'] == 'UTR5')]

In [42]:
## Filter exonic and synonymous variants
coding_synonymous = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'synonymous SNV')]

In [43]:
# Filter exonic and non-synonymous variants
coding_nonsynonymous = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'nonsynonymous SNV')]
coding_nonsynonymous

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo5139,Otherinfo5140,Otherinfo5141,Otherinfo5142,Otherinfo5143,Otherinfo5144,Otherinfo5145,Otherinfo5146,Otherinfo5147,Otherinfo5148
371,3,132454147,132454147,T,A,exonic,DNAJC13,.,nonsynonymous SNV,"DNAJC13:NM_001329126:exon9:c.T922A:p.S308T,DNA...",...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
398,3,132457349,132457349,T,C,exonic,DNAJC13,.,nonsynonymous SNV,"DNAJC13:NM_001329126:exon13:c.T1430C:p.M477T,D...",...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
571,3,132477807,132477807,G,A,exonic,DNAJC13,.,nonsynonymous SNV,"DNAJC13:NM_015268:exon23:c.G2464A:p.A822T,DNAJ...",...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
629,3,132484610,132484610,C,T,exonic,DNAJC13,.,nonsynonymous SNV,"DNAJC13:NM_015268:exon29:c.C3205T:p.R1069W,DNA...",...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
703,3,132492604,132492604,A,G,exonic,DNAJC13,.,nonsynonymous SNV,"DNAJC13:NM_015268:exon33:c.A3814G:p.I1272V,DNA...",...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
717,3,132494190,132494190,A,G,exonic,DNAJC13,.,nonsynonymous SNV,"DNAJC13:NM_015268:exon34:c.A3872G:p.E1291G,DNA...",...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
726,3,132495159,132495159,A,G,exonic,DNAJC13,.,nonsynonymous SNV,"DNAJC13:NM_015268:exon35:c.A4013G:p.E1338G,DNA...",...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
743,3,132496652,132496652,G,A,exonic,DNAJC13,.,nonsynonymous SNV,"DNAJC13:NM_015268:exon36:c.G4145A:p.R1382H,DNA...",...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
774,3,132499231,132499231,C,T,exonic,DNAJC13,.,nonsynonymous SNV,"DNAJC13:NM_015268:exon37:c.C4262T:p.A1421V,DNA...",...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
780,3,132499777,132499777,G,A,exonic,DNAJC13,.,nonsynonymous SNV,"DNAJC13:NM_015268:exon38:c.G4385A:p.R1462H,DNA...",...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0


In [44]:
totalVariants = len(gene.axes[0])
totalIntronic = len(intronic.axes[0])
totalUTR3 = len(utr3.axes[0])
totalUTR5 = len(utr5.axes[0])
totalExonicSyn = len(coding_synonymous.axes[0])
totalExonicNonSyn = len(coding_nonsynonymous.axes[0])

In [45]:
print("Total Variants: ", totalVariants)
print("Intronic:", totalIntronic)
print("UTR3:", totalUTR3)
print("UTR5:", totalUTR5)
print("Exonic Syn:", totalExonicSyn)
print("Exonic NonSyn:", totalExonicNonSyn)

Total Variants:  1179
Intronic: 1140
UTR3: 8
UTR5: 3
Exonic Syn: 6
Exonic NonSyn: 22


In [46]:
# Save in PLINK format 
variants_toKeep = coding_nonsynonymous[['Chr', 'Start', 'End', 'Gene.refGene']].copy()
variants_toKeep.to_csv(f'{WORK_DIR}/{ancestry}_DNAJC13.all_coding_nonsyn.variantstoKeep', sep="\t", index=False, header=False)
variants_toKeep

,Chr,Start,End,Gene.refGene
371,3,132454147,132454147,DNAJC13
398,3,132457349,132457349,DNAJC13
571,3,132477807,132477807,DNAJC13
629,3,132484610,132484610,DNAJC13
703,3,132492604,132492604,DNAJC13
717,3,132494190,132494190,DNAJC13
726,3,132495159,132495159,DNAJC13
743,3,132496652,132496652,DNAJC13
774,3,132499231,132499231,DNAJC13
780,3,132499777,132499777,DNAJC13


In [47]:
## check to make sure file was created and saved
! ls {WORK_DIR}

chr3_EAS_release7.log			      EAS_DNAJC13.log
chr3_EAS_release7.pgen			      EAS_DNAJC13.vcf.gz
chr3_EAS_release7.psam			      EAS_DNAJC13.vcf.gz.tbi
chr3_EAS_release7.pvar			      EAS_release7.eigenvec
EAS_covariate_file.txt			      EAS_release7.related
EAS_DNAJC13.all_coding_nonsyn.variantstoKeep  EAS.samplestoKeep
EAS_DNAJC13.annovar.avinput		      master_key_release7_final.csv
EAS_DNAJC13.annovar.hg38_multianno.txt	      refFlat_HG38_all_chr.txt
EAS_DNAJC13.annovar.hg38_multianno.vcf


In [ ]:
# Save to workspace bucket (move from VM to workspace bucket)
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_DNAJC13.all_coding_nonsyn.variantstoKeep {WORKSPACE_BUCKET}//{WORKSPACE_DIR}/')

In [ ]:
## Check workspace bucket
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} ls {WORKSPACE_BUCKET}/{WORKSPACE_DIR}/')

## Burden Analyses using RVTests

In [50]:
# Convert the files from Plink 2.0 to Plink 1.9 format

! /home/jupyter/tools/plink2 --pfile {WORK_DIR}/chr3_{ancestry}_release7 \
--make-bed \
--out {WORK_DIR}/chr3_{ancestry}_release7

PLINK v2.00a6LM AVX2 AMD (4 Jul 2024)          www.cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to 202407_BURDEN_EAS_R7/chr3_EAS_release7.log.
Options in effect:
  --make-bed
  --out 202407_BURDEN_EAS_R7/chr3_EAS_release7
  --pfile 202407_BURDEN_EAS_R7/chr3_EAS_release7

Start time: Wed Jul 10 00:56:04 2024
7450 MiB RAM detected, ~4495 available; reserving 3725 MiB for main workspace.
Using up to 2 compute threads.
5136 samples (1868 females, 3268 males; 5136 founders) loaded from
202407_BURDEN_EAS_R7/chr3_EAS_release7.psam.
3353374 variants loaded from 202407_BURDEN_EAS_R7/chr3_EAS_release7.pvar.
1 binary phenotype loaded (2644 cases, 2452 controls).
Writing 202407_BURDEN_EAS_R7/chr3_EAS_release7.fam ... done.
Writing 202407_BURDEN_EAS_R7/chr3_EAS_release7.bim ... done.
Writing 202407_BURDEN_EAS_R7/chr3_EAS_release7.bed ... 11131517192123252729313335373941424446485052545658606264666870727476788082848587899193959799d

In [51]:
## extract variants
    # later do based on class and frequency

! /home/jupyter/tools/plink \
--bfile {WORK_DIR}/chr3_{ancestry}_release7 \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--extract range {WORK_DIR}/{ancestry}_DNAJC13.all_coding_nonsyn.variantstoKeep \
--recode vcf-iid \
--out {WORK_DIR}/{ancestry}_DNAJC13.coding

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to 202407_BURDEN_EAS_R7/EAS_DNAJC13.coding.log.
Options in effect:
  --bfile 202407_BURDEN_EAS_R7/chr3_EAS_release7
  --extract range 202407_BURDEN_EAS_R7/EAS_DNAJC13.all_coding_nonsyn.variantstoKeep
  --keep 202407_BURDEN_EAS_R7/EAS.samplestoKeep
  --out 202407_BURDEN_EAS_R7/EAS_DNAJC13.coding
  --recode vcf-iid

7450 MB RAM detected; reserving 3725 MB for main workspace.
3353374 variants loaded from .bim file.
5136 people (3268 males, 1868 females) loaded from .fam.
5096 phenotype values loaded from .fam.
--extract range: 3353352 variants excluded.
--extract range: 22 variants remaining.
--keep: 5135 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 5135 founders and 0 nonfounders present.
Calculating allele frequencies... 1011121314151617181920212223242526272829303132

In [52]:
### Bgzip and Tabix (zip and index the file)
! bgzip -f {WORK_DIR}/{ancestry}_DNAJC13.coding.vcf
! tabix -f -p vcf {WORK_DIR}/{ancestry}_DNAJC13.coding.vcf.gz

In [53]:
## RVtests with covariates 
! /home/jupyter/tools/rvtests/executable/rvtest --noweb --hide-covar \
--out {WORK_DIR}/{ancestry}_DNAJC13.burden.coding \
--kernel skat,skato \
--inVcf {WORK_DIR}/{ancestry}_DNAJC13.coding.vcf.gz \
--pheno {WORK_DIR}/{ancestry}_covariate_file.txt \
--mpheno 4 \
--gene DNAJC13 \
--geneFile {WORK_DIR}/refFlat_HG38_all_chr.txt \
--covar {WORK_DIR}/{ancestry}_covariate_file.txt \
--covar-name SEX,PC1,PC2,PC3,PC4 #[WARN]Covariate [ PC5 ] has strong correlation [ r^2 = 1 ] with the response!


# --out : Name of output 
# --burden cmc --kernel skato: tests to run 
# --inVcf : VCF file 
# --gene: gene name (if only looking at one or a few)
# --geneFile refFlat.txt
# --pheno :  covar file
# --mpheno : # column that has phenotype information
# --pheno-name : column name with phenotype in file
# --covar : covar file
# --freqUpper : optional, MAF cut-off
# --covar-name : covariates, listed by column name, separated by commas (no spaces between commas)
## 0=controls; 1=cases

Thank you for using rvtests (version: 20190205, git: c86e589efef15382603300dc7f4c3394c82d69b8)
  For documentations, refer to http://zhanxw.github.io/rvtests/
  For questions and comments, plase send to Xiaowei Zhan <zhanxw@umich.edu>
  For bugs and feature requests, please submit at: https://github.com/zhanxw/rvtests/issues

The following parameters are available.  Ones with "[]" are in effect:

Available Options
      Basic Input/Output:
                          --inVcf [202407_BURDEN_EAS_R7/EAS_DNAJC13.coding.vcf.gz]
                          --inBgen [], --inBgenSample [], --inKgg []
                          --out [202407_BURDEN_EAS_R7/EAS_DNAJC13.burden.coding]
                          --outputRaw
       Specify Covariate: --covar [202407_BURDEN_EAS_R7/EAS_covariate_file.txt]
                          --covar-name [SEX,PC1,PC2,PC3,PC4], --sex
       Specify Phenotype: --pheno [202407_BURDEN_EAS_R7/EAS_covariate_file.txt]
                          --inverseNormal, --useResidualA

In [54]:
## look at results 
! cat {WORK_DIR}/{ancestry}_DNAJC13.burden.coding.Skat.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	Pvalue	NumPerm	ActualPerm	Stat	NumGreater	NumEqual	PermPvalue
DNAJC13	3:132417501-132539032,3:132417501-132539032	4891	22	17	0.113714	0.999986	10000	1000	0.113714	1000	0	1


In [55]:
! cat {WORK_DIR}/{ancestry}_DNAJC13.burden.coding.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
DNAJC13	3:132417501-132539032,3:132417501-132539032	4891	22	17	653.777	1	1


In [56]:
## check to make sure file was created and saved
! ls {WORK_DIR}

chr3_EAS_release7.bed
chr3_EAS_release7.bim
chr3_EAS_release7.fam
chr3_EAS_release7.log
chr3_EAS_release7.pgen
chr3_EAS_release7.psam
chr3_EAS_release7.pvar
EAS_covariate_file.txt
EAS_DNAJC13.all_coding_nonsyn.variantstoKeep
EAS_DNAJC13.annovar.avinput
EAS_DNAJC13.annovar.hg38_multianno.txt
EAS_DNAJC13.annovar.hg38_multianno.vcf
EAS_DNAJC13.burden.coding.log
EAS_DNAJC13.burden.coding.Skat.assoc
EAS_DNAJC13.burden.coding.SkatO.assoc
EAS_DNAJC13.coding.log
EAS_DNAJC13.coding.vcf.gz
EAS_DNAJC13.coding.vcf.gz.tbi
EAS_DNAJC13.log
EAS_DNAJC13.vcf.gz
EAS_DNAJC13.vcf.gz.tbi
EAS_release7.eigenvec
EAS_release7.related
EAS.samplestoKeep
master_key_release7_final.csv
refFlat_HG38_all_chr.txt


In [ ]:
# Save to workspace bucket (move from VM to workspace bucket)
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_DNAJC13.burden.coding.*.assoc {WORKSPACE_BUCKET}/{WORKSPACE_DIR}/')

In [ ]:
## Check workspace bucket
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} ls {WORKSPACE_BUCKET}/{WORKSPACE_DIR}/')

# Case/Control Frequencies

In [59]:
## extract variants

! /home/jupyter/tools/plink \
--bfile {WORK_DIR}/chr3_{ancestry}_release7 \
--extract range {WORK_DIR}/{ancestry}_DNAJC13.all_coding_nonsyn.variantstoKeep \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--assoc \
--out {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn --allow-no-sex --ci 0.95

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to 202407_BURDEN_EAS_R7/EAS_DNAJC13.coding_nonsyn.log.
Options in effect:
  --allow-no-sex
  --assoc
  --bfile 202407_BURDEN_EAS_R7/chr3_EAS_release7
  --ci 0.95
  --extract range 202407_BURDEN_EAS_R7/EAS_DNAJC13.all_coding_nonsyn.variantstoKeep
  --keep 202407_BURDEN_EAS_R7/EAS.samplestoKeep
  --out 202407_BURDEN_EAS_R7/EAS_DNAJC13.coding_nonsyn

7450 MB RAM detected; reserving 3725 MB for main workspace.
3353374 variants loaded from .bim file.
5136 people (3268 males, 1868 females) loaded from .fam.
5096 phenotype values loaded from .fam.
--extract range: 3353352 variants excluded.
--extract range: 22 variants remaining.
--keep: 5135 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 5135 founders and 0 nonfounders present.
Calculating allele frequencies... 101112131415

In [60]:
! cat {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.assoc

 CHR                  SNP         BP   A1      F_A      F_U   A2        CHISQ            P           OR           SE          L95          U95 
   3   chr3:132454147:T:A  132454147    A        0 0.000204    T        1.078       0.2991            0          inf            0          nan 
   3   chr3:132457349:T:C  132457349    C        0 0.0002042    T        1.077       0.2993            0          inf            0          nan 
   3   chr3:132477807:G:A  132477807    A 0.0009509 0.001847    G        1.472        0.225       0.5143       0.5581       0.1722        1.536 
   3   chr3:132484610:C:T  132484610    T        0 0.0002039    C        1.078       0.2992            0          inf            0          nan 
   3   chr3:132492604:A:G  132492604    G 0.0003784        0    A        1.856       0.1731           NA           NA           NA           NA 
   3   chr3:132494190:A:G  132494190    G 0.0003785        0    A        1.857        0.173           NA           NA           NA  

In [61]:
assoc = pd.read_csv(f'{WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.assoc', delim_whitespace=True, usecols=['SNP', 'A1', 'A2', 'F_A', 'F_U', 'P', 'OR', 'L95', 'U95'])
assoc

,SNP,A1,F_A,F_U,A2,P,OR,L95,U95
0,chr3:132454147:T:A,A,0.000000,0.000204,T,2.991000e-01,0.0000,0.00000,NaN
1,chr3:132457349:T:C,C,0.000000,0.000204,T,2.993000e-01,0.0000,0.00000,NaN
2,chr3:132477807:G:A,A,0.000951,0.001847,G,2.250000e-01,0.5143,0.17220,1.5360
3,chr3:132484610:C:T,T,0.000000,0.000204,C,2.992000e-01,0.0000,0.00000,NaN
4,chr3:132492604:A:G,G,0.000378,0.000000,A,1.731000e-01,NaN,NaN,NaN
5,chr3:132494190:A:G,G,0.000378,0.000000,A,1.730000e-01,NaN,NaN,NaN
6,chr3:132495159:A:G,G,0.000000,0.000000,A,NaN,NaN,NaN,NaN
7,chr3:132496652:G:A,A,0.000000,0.000000,G,NaN,NaN,NaN,NaN
8,chr3:132499231:C:T,T,0.000000,0.000408,C,1.420000e-01,0.0000,0.00000,NaN
9,chr3:132499777:G:A,A,0.000189,0.001020,G,8.425000e-02,0.1854,0.02165,1.5870


In [62]:
## repeat association study applying multiple testing corrections for the raw p-values (like Bonferroni post-hoc)
! /home/jupyter/tools/plink \
--bfile {WORK_DIR}/chr3_{ancestry}_release7 \
--extract range {WORK_DIR}/{ancestry}_DNAJC13.all_coding_nonsyn.variantstoKeep \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--assoc \
--out {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn --allow-no-sex --adjust

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to 202407_BURDEN_EAS_R7/EAS_DNAJC13.coding_nonsyn.log.
Options in effect:
  --adjust
  --allow-no-sex
  --assoc
  --bfile 202407_BURDEN_EAS_R7/chr3_EAS_release7
  --extract range 202407_BURDEN_EAS_R7/EAS_DNAJC13.all_coding_nonsyn.variantstoKeep
  --keep 202407_BURDEN_EAS_R7/EAS.samplestoKeep
  --out 202407_BURDEN_EAS_R7/EAS_DNAJC13.coding_nonsyn

7450 MB RAM detected; reserving 3725 MB for main workspace.
3353374 variants loaded from .bim file.
5136 people (3268 males, 1868 females) loaded from .fam.
5096 phenotype values loaded from .fam.
--extract range: 3353352 variants excluded.
--extract range: 22 variants remaining.
--keep: 5135 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 5135 founders and 0 nonfounders present.
Calculating allele frequencies... 1011121314151

In [63]:
! cat {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.assoc.adjusted

 CHR                  SNP      UNADJ         GC       BONF       HOLM   SIDAK_SS   SIDAK_SD     FDR_BH     FDR_BY
   3   chr3:132499779:G:T  1.635e-14  5.913e-07  2.942e-13  2.942e-13  2.942e-13  2.942e-13  2.942e-13  1.028e-12 
   3   chr3:132499777:G:A    0.08425     0.2613          1          1     0.7949      0.776     0.4919          1 
   3   chr3:132499231:C:T      0.142     0.3394          1          1     0.9365     0.9137     0.4919          1 
   3   chr3:132502298:C:T     0.1535     0.3531          1          1     0.9502     0.9179     0.4919          1 
   3   chr3:132494190:A:G      0.173     0.3754          1          1     0.9673       0.93     0.4919          1 
   3   chr3:132492604:A:G     0.1731     0.3755          1          1     0.9673       0.93     0.4919          1 
   3   chr3:132477807:G:A      0.225     0.4299          1          1     0.9898     0.9531     0.4919          1 
   3   chr3:132454147:T:A     0.2991     0.4994          1          1     0.9983 

In [64]:
assoc_adjusted = pd.read_csv(f'{WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.assoc.adjusted', delim_whitespace=True, usecols=['SNP', 'BONF'])
assoc_adjusted

,SNP,BONF
0,chr3:132499779:G:T,2.942000e-13
1,chr3:132499777:G:A,1.000000e+00
2,chr3:132499231:C:T,1.000000e+00
3,chr3:132502298:C:T,1.000000e+00
4,chr3:132494190:A:G,1.000000e+00
5,chr3:132492604:A:G,1.000000e+00
6,chr3:132477807:G:A,1.000000e+00
7,chr3:132454147:T:A,1.000000e+00
8,chr3:132484610:C:T,1.000000e+00
9,chr3:132457349:T:C,1.000000e+00


In [65]:
! /home/jupyter/tools/plink \
--bfile {WORK_DIR}/chr3_{ancestry}_release7 \
--extract range {WORK_DIR}/{ancestry}_DNAJC13.all_coding_nonsyn.variantstoKeep \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--freq \
--out {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to 202407_BURDEN_EAS_R7/EAS_DNAJC13.coding_nonsyn.log.
Options in effect:
  --bfile 202407_BURDEN_EAS_R7/chr3_EAS_release7
  --extract range 202407_BURDEN_EAS_R7/EAS_DNAJC13.all_coding_nonsyn.variantstoKeep
  --freq
  --keep 202407_BURDEN_EAS_R7/EAS.samplestoKeep
  --out 202407_BURDEN_EAS_R7/EAS_DNAJC13.coding_nonsyn

7450 MB RAM detected; reserving 3725 MB for main workspace.
3353374 variants loaded from .bim file.
5136 people (3268 males, 1868 females) loaded from .fam.
5096 phenotype values loaded from .fam.
--extract range: 3353352 variants excluded.
--extract range: 22 variants remaining.
--keep: 5135 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 5135 founders and 0 nonfounders present.
Calculating allele frequencies... 101112131415161718192021222324252627282930

In [66]:
! cat {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.frq

 CHR                  SNP   A1   A2          MAF  NCHROBS
   3   chr3:132454147:T:A    A    T    9.741e-05    10266
   3   chr3:132457349:T:C    C    T    9.752e-05    10254
   3   chr3:132477807:G:A    A    G     0.001371    10210
   3   chr3:132484610:C:T    T    C    9.739e-05    10268
   3   chr3:132492604:A:G    G    A    0.0001947    10270
   3   chr3:132494190:A:G    G    A    0.0001948    10268
   3   chr3:132495159:A:G    G    A            0    10262
   3   chr3:132496652:G:A    A    G            0    10254
   3   chr3:132499231:C:T    T    C    0.0001947    10270
   3   chr3:132499777:G:A    A    G    0.0005842    10270
   3   chr3:132499779:G:T    G    T       0.1241     9242
   3   chr3:132502295:C:T    T    C    0.0003975    10064
   3   chr3:132502298:C:T    T    C    0.0004873    10260
   3   chr3:132503217:G:A    A    G    9.748e-05    10258
   3   chr3:132507256:A:G    G    A     0.001169    10268
   3   chr3:132511239:A:G    G    A    9.762e-05    10244
   3   chr3:13

In [67]:
freq = pd.read_csv(f'{WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.frq', delim_whitespace=True, usecols=['SNP', 'MAF'])
freq.head()

,SNP,MAF
0,chr3:132454147:T:A,0.000097
1,chr3:132457349:T:C,0.000098
2,chr3:132477807:G:A,0.001371
3,chr3:132484610:C:T,0.000097
4,chr3:132492604:A:G,0.000195


In [68]:
! /home/jupyter/tools/plink \
--bfile {WORK_DIR}/chr3_{ancestry}_release7 \
--extract range {WORK_DIR}/{ancestry}_DNAJC13.all_coding_nonsyn.variantstoKeep \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--freq case-control \
--out {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to 202407_BURDEN_EAS_R7/EAS_DNAJC13.coding_nonsyn.log.
Options in effect:
  --bfile 202407_BURDEN_EAS_R7/chr3_EAS_release7
  --extract range 202407_BURDEN_EAS_R7/EAS_DNAJC13.all_coding_nonsyn.variantstoKeep
  --freq case-control
  --keep 202407_BURDEN_EAS_R7/EAS.samplestoKeep
  --out 202407_BURDEN_EAS_R7/EAS_DNAJC13.coding_nonsyn

7450 MB RAM detected; reserving 3725 MB for main workspace.
3353374 variants loaded from .bim file.
5136 people (3268 males, 1868 females) loaded from .fam.
5096 phenotype values loaded from .fam.
--extract range: 3353352 variants excluded.
--extract range: 22 variants remaining.
--keep: 5135 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 5135 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232

In [69]:
! cat {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.frq.cc

 CHR                  SNP   A1   A2        MAF_A        MAF_U  NCHROBS_A  NCHROBS_U
   3   chr3:132454147:T:A    A    T            0     0.000204       5284       4902
   3   chr3:132457349:T:C    C    T            0    0.0002042       5276       4898
   3   chr3:132477807:G:A    A    G    0.0009509     0.001847       5258       4872
   3   chr3:132484610:C:T    T    C            0    0.0002039       5284       4904
   3   chr3:132492604:A:G    G    A    0.0003784            0       5286       4904
   3   chr3:132494190:A:G    G    A    0.0003785            0       5284       4904
   3   chr3:132495159:A:G    G    A            0            0       5280       4902
   3   chr3:132496652:G:A    A    G            0            0       5274       4900
   3   chr3:132499231:C:T    T    C            0    0.0004078       5286       4904
   3   chr3:132499777:G:A    A    G    0.0001892      0.00102       5286       4904
   3   chr3:132499779:G:T    G    T      0.09946       0.1525       4836    

In [70]:
freq_cc = pd.read_csv(f'{WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.frq.cc', delim_whitespace=True, usecols=['SNP', 'MAF_A', 'MAF_U'])
freq_cc.head()

,SNP,MAF_A,MAF_U
0,chr3:132454147:T:A,0.000000,0.000204
1,chr3:132457349:T:C,0.000000,0.000204
2,chr3:132477807:G:A,0.000951,0.001847
3,chr3:132484610:C:T,0.000000,0.000204
4,chr3:132492604:A:G,0.000378,0.000000


In [71]:
all_freq = pd.merge(freq, freq_cc, on="SNP", how="left")
all_freq

,SNP,MAF,MAF_A,MAF_U
0,chr3:132454147:T:A,0.000097,0.000000,0.000204
1,chr3:132457349:T:C,0.000098,0.000000,0.000204
2,chr3:132477807:G:A,0.001371,0.000951,0.001847
3,chr3:132484610:C:T,0.000097,0.000000,0.000204
4,chr3:132492604:A:G,0.000195,0.000378,0.000000
5,chr3:132494190:A:G,0.000195,0.000378,0.000000
6,chr3:132495159:A:G,0.000000,0.000000,0.000000
7,chr3:132496652:G:A,0.000000,0.000000,0.000000
8,chr3:132499231:C:T,0.000195,0.000000,0.000408
9,chr3:132499777:G:A,0.000584,0.000189,0.001020


In [72]:
assoc_and_freq = pd.merge(all_freq, assoc, on="SNP", how="left")
assoc_and_freq

,SNP,MAF,MAF_A,MAF_U,A1,F_A,F_U,A2,P,OR,L95,U95
0,chr3:132454147:T:A,0.000097,0.000000,0.000204,A,0.000000,0.000204,T,2.991000e-01,0.0000,0.00000,NaN
1,chr3:132457349:T:C,0.000098,0.000000,0.000204,C,0.000000,0.000204,T,2.993000e-01,0.0000,0.00000,NaN
2,chr3:132477807:G:A,0.001371,0.000951,0.001847,A,0.000951,0.001847,G,2.250000e-01,0.5143,0.17220,1.5360
3,chr3:132484610:C:T,0.000097,0.000000,0.000204,T,0.000000,0.000204,C,2.992000e-01,0.0000,0.00000,NaN
4,chr3:132492604:A:G,0.000195,0.000378,0.000000,G,0.000378,0.000000,A,1.731000e-01,NaN,NaN,NaN
5,chr3:132494190:A:G,0.000195,0.000378,0.000000,G,0.000378,0.000000,A,1.730000e-01,NaN,NaN,NaN
6,chr3:132495159:A:G,0.000000,0.000000,0.000000,G,0.000000,0.000000,A,NaN,NaN,NaN,NaN
7,chr3:132496652:G:A,0.000000,0.000000,0.000000,A,0.000000,0.000000,G,NaN,NaN,NaN,NaN
8,chr3:132499231:C:T,0.000195,0.000000,0.000408,T,0.000000,0.000408,C,1.420000e-01,0.0000,0.00000,NaN
9,chr3:132499777:G:A,0.000584,0.000189,0.001020,A,0.000189,0.001020,G,8.425000e-02,0.1854,0.02165,1.5870


In [73]:
# Merge the results
assoc_and_freq = pd.merge(assoc_and_freq, assoc_adjusted, on="SNP", how="left")
assoc_and_freq.head()

,SNP,MAF,MAF_A,MAF_U,A1,F_A,F_U,A2,P,OR,L95,U95,BONF
0,chr3:132454147:T:A,0.000097,0.000000,0.000204,A,0.000000,0.000204,T,0.2991,0.0000,0.0000,NaN,1.0
1,chr3:132457349:T:C,0.000098,0.000000,0.000204,C,0.000000,0.000204,T,0.2993,0.0000,0.0000,NaN,1.0
2,chr3:132477807:G:A,0.001371,0.000951,0.001847,A,0.000951,0.001847,G,0.2250,0.5143,0.1722,1.536,1.0
3,chr3:132484610:C:T,0.000097,0.000000,0.000204,T,0.000000,0.000204,C,0.2992,0.0000,0.0000,NaN,1.0
4,chr3:132492604:A:G,0.000195,0.000378,0.000000,G,0.000378,0.000000,A,0.1731,NaN,NaN,NaN,1.0


In [74]:
! /home/jupyter/tools/plink \
--bfile {WORK_DIR}/chr3_{ancestry}_release7 \
--extract range {WORK_DIR}/{ancestry}_DNAJC13.all_coding_nonsyn.variantstoKeep \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--recode A \
--out {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to 202407_BURDEN_EAS_R7/EAS_DNAJC13.coding_nonsyn.log.
Options in effect:
  --bfile 202407_BURDEN_EAS_R7/chr3_EAS_release7
  --extract range 202407_BURDEN_EAS_R7/EAS_DNAJC13.all_coding_nonsyn.variantstoKeep
  --keep 202407_BURDEN_EAS_R7/EAS.samplestoKeep
  --out 202407_BURDEN_EAS_R7/EAS_DNAJC13.coding_nonsyn
  --recode A

7450 MB RAM detected; reserving 3725 MB for main workspace.
3353374 variants loaded from .bim file.
5136 people (3268 males, 1868 females) loaded from .fam.
5096 phenotype values loaded from .fam.
--extract range: 3353352 variants excluded.
--extract range: 22 variants remaining.
--keep: 5135 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 5135 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728

In [ ]:
recode = pd.read_csv(f'{WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.raw', delim_whitespace=True)
recode.head()

In [76]:
# Make a list from the column names
column_names = recode.columns.tolist()

# Drop the first 6 columns to keep the variants 
variants = column_names[6:]

print(f'Number of variants in {ancestry} for DNAJC13: {len(variants)}')
variants

Number of variants in EAS for DNAJC13: 22


['chr3:132454147:T:A_A',
 'chr3:132457349:T:C_C',
 'chr3:132477807:G:A_A',
 'chr3:132484610:C:T_T',
 'chr3:132492604:A:G_G',
 'chr3:132494190:A:G_G',
 'chr3:132495159:A:G_G',
 'chr3:132496652:G:A_A',
 'chr3:132499231:C:T_T',
 'chr3:132499777:G:A_A',
 'chr3:132499779:G:T_G',
 'chr3:132502295:C:T_T',
 'chr3:132502298:C:T_T',
 'chr3:132503217:G:A_A',
 'chr3:132507256:A:G_G',
 'chr3:132511239:A:G_G',
 'chr3:132516743:C:T_T',
 'chr3:132516789:G:C_C',
 'chr3:132525761:G:A_A',
 'chr3:132538179:C:T_T',
 'chr3:132538225:G:A_A',
 'chr3:132538272:A:G_G']

In [77]:
# Pre-filter the dataset
cases_data = recode[recode['PHENOTYPE'] == 2]
controls_data = recode[recode['PHENOTYPE'] == 1]

results = []

for variant in variants:
    # For cases
    hom_cases = cases_data[cases_data[variant] == 2].shape[0]
    het_cases = cases_data[cases_data[variant] == 1].shape[0]
    total_cases = cases_data.shape[0]
#    freq_cases = (hom_cases + het_cases) / total_cases

    # For controls
    hom_controls = controls_data[controls_data[variant] == 2].shape[0]
    het_controls = controls_data[controls_data[variant] == 1].shape[0]
    total_controls = controls_data.shape[0]
#    freq_controls = (hom_controls + het_controls) / total_controls

    results.append({
        'Variant': variant,
        'Hom Cases': hom_cases,
        'Het Cases': het_cases,
        'Total Cases': total_cases,
#        'Freq in Cases': freq_cases,
        'Hom Controls': hom_controls,
        'Het Controls': het_controls,
        'Total Controls': total_controls,
#        'Freq in Controls': freq_controls
    })

# Return
df_results = pd.DataFrame(results)
df_results['SNP'] = df_results['Variant'].apply(lambda x: x.rsplit('_', 1)[0])

df_results

,Variant,Hom Cases,Het Cases,Total Cases,Hom Controls,Het Controls,Total Controls,SNP
0,chr3:132454147:T:A_A,0,0,2643,0,1,2452,chr3:132454147:T:A
1,chr3:132457349:T:C_C,0,0,2643,0,1,2452,chr3:132457349:T:C
2,chr3:132477807:G:A_A,0,5,2643,0,9,2452,chr3:132477807:G:A
3,chr3:132484610:C:T_T,0,0,2643,0,1,2452,chr3:132484610:C:T
4,chr3:132492604:A:G_G,0,2,2643,0,0,2452,chr3:132492604:A:G
5,chr3:132494190:A:G_G,0,2,2643,0,0,2452,chr3:132494190:A:G
6,chr3:132495159:A:G_G,0,0,2643,0,0,2452,chr3:132495159:A:G
7,chr3:132496652:G:A_A,0,0,2643,0,0,2452,chr3:132496652:G:A
8,chr3:132499231:C:T_T,0,0,2643,0,2,2452,chr3:132499231:C:T
9,chr3:132499777:G:A_A,0,1,2643,0,5,2452,chr3:132499777:G:A


In [78]:
## Merge with assoc results 
full_results = pd.merge(assoc_and_freq, df_results, on="SNP", how="left")

In [79]:
clean_full_results = full_results[['SNP', 'A1', 'A2', 'F_A', 'F_U', 'P', 'OR','L95', 'U95', 'BONF', 
                                   'MAF', 'MAF_A', 'MAF_U', 
                                   'Hom Cases', 'Het Cases', 'Total Cases', 
                                   'Hom Controls','Het Controls', 'Total Controls']].copy()

In [80]:
print(clean_full_results.shape)
print(f'No. of SNPs: {clean_full_results.shape[0]}')
clean_full_results

(22, 19)
No. of SNPs: 22


,SNP,A1,A2,F_A,F_U,P,OR,L95,U95,BONF,MAF,MAF_A,MAF_U,Hom Cases,Het Cases,Total Cases,Hom Controls,Het Controls,Total Controls
0,chr3:132454147:T:A,A,T,0.000000,0.000204,2.991000e-01,0.0000,0.00000,NaN,1.000000e+00,0.000097,0.000000,0.000204,0,0,2643,0,1,2452
1,chr3:132457349:T:C,C,T,0.000000,0.000204,2.993000e-01,0.0000,0.00000,NaN,1.000000e+00,0.000098,0.000000,0.000204,0,0,2643,0,1,2452
2,chr3:132477807:G:A,A,G,0.000951,0.001847,2.250000e-01,0.5143,0.17220,1.5360,1.000000e+00,0.001371,0.000951,0.001847,0,5,2643,0,9,2452
3,chr3:132484610:C:T,T,C,0.000000,0.000204,2.992000e-01,0.0000,0.00000,NaN,1.000000e+00,0.000097,0.000000,0.000204,0,0,2643,0,1,2452
4,chr3:132492604:A:G,G,A,0.000378,0.000000,1.731000e-01,NaN,NaN,NaN,1.000000e+00,0.000195,0.000378,0.000000,0,2,2643,0,0,2452
5,chr3:132494190:A:G,G,A,0.000378,0.000000,1.730000e-01,NaN,NaN,NaN,1.000000e+00,0.000195,0.000378,0.000000,0,2,2643,0,0,2452
6,chr3:132495159:A:G,G,A,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0,0,2643,0,0,2452
7,chr3:132496652:G:A,A,G,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0,0,2643,0,0,2452
8,chr3:132499231:C:T,T,C,0.000000,0.000408,1.420000e-01,0.0000,0.00000,NaN,1.000000e+00,0.000195,0.000000,0.000408,0,0,2643,0,2,2452
9,chr3:132499777:G:A,A,G,0.000189,0.001020,8.425000e-02,0.1854,0.02165,1.5870,1.000000e+00,0.000584,0.000189,0.001020,0,1,2643,0,5,2452


In [81]:
# Look at significant SNPs, if any 
sig_freq = clean_full_results[clean_full_results['P']<0.05]
sig_snps = sig_freq['SNP'].tolist()
sig_snps

['chr3:132499779:G:T']

In [82]:
# Look at significant SNPs, if any 
sig_df_results = clean_full_results[clean_full_results['SNP'].isin(sig_snps)]
sig_df_results

,SNP,A1,A2,F_A,F_U,P,OR,L95,U95,BONF,MAF,MAF_A,MAF_U,Hom Cases,Het Cases,Total Cases,Hom Controls,Het Controls,Total Controls
10,chr3:132499779:G:T,G,T,0.09946,0.1525,1.635000e-14,0.6138,0.5415,0.6959,2.942000e-13,0.1241,0.09946,0.1525,24,433,2643,51,558,2452


In [83]:
for snp in sig_snps:
    ! grep {snp} {WORK_DIR}/chr3_{ancestry}_release7.pvar

3	132499779	chr3:132499779:G:T	G	T	IMPUTED;AF=0.855382;MAF=0.144618;AVG_CS=0.978315;R2=0.852045


In [84]:
# Save files to VM
clean_full_results.to_csv(f'{WORK_DIR}/{ancestry}_DNAJC13.fullVariantInformation.txt', sep="\t", index=False)
sig_df_results.to_csv(f'{WORK_DIR}/{ancestry}_DNAJC13.SignificantVariantInformation.txt' , sep="\t", index=False)

In [ ]:
# Save to workspace bucket (move from VM to workspace bucket)
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_DNAJC13.annovar.hg38_multianno.txt {WORKSPACE_BUCKET}/{WORKSPACE_DIR}/')
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_DNAJC13.fullVariantInformation.txt {WORKSPACE_BUCKET}/{WORKSPACE_DIR}/')
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_DNAJC13.SignificantVariantInformation.txt {WORKSPACE_BUCKET}/{WORKSPACE_DIR}/')

shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.raw {WORKSPACE_BUCKET}/{WORKSPACE_DIR}/')
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.frq.cc {WORKSPACE_BUCKET}/{WORKSPACE_DIR}/')
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.frq {WORKSPACE_BUCKET}/{WORKSPACE_DIR}/')
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.assoc {WORKSPACE_BUCKET}/{WORKSPACE_DIR}/')
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.assoc.adjusted {WORKSPACE_BUCKET}/{WORKSPACE_DIR}/')

In [ ]:
## Check workspace bucket
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} ls {WORKSPACE_BUCKET}/{WORKSPACE_DIR}/')